## Structured output

The research endpoint lets you provide an `output_schema` so the research results are returned in the exact structure you specify. The schema must follow this format.


You must provide a `"type"` and `"description"` for every property. `"type"` must be one of `"object"`, `"string"`, `"integer"`, `"number"` or `"array"`.

Each `"object"` property must define its own `"properties"` and a `"description"`, and each `"array"` property must define `"items"` and a `"description"`.


In [ ]:
%pip install -q tavily-python

In [ ]:
import getpass
import os
import httpx
import time
from IPython.display import display, JSON
from tavily import TavilyClient

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [ ]:
input = "Create a detailed company research report about Nvidia. Focus on leadership, financials, AI initiatives, infrastructure, and include a deep two-year GPU sales analysis."
model = "pro"
output_schema = {
    "properties": {
        "company_name": {
            "type": "string",
            "description": "Company name"
        },
        "ceo": {
            "type": "string",
            "description": "Chief Executive Officer"
        },
        "cto": {
            "type": "string",
            "description": "Chief Technology Officer"
        },
        "compute_and_cloud_offerings": {
            "type": "string",
            "description": "Compute platforms, AI chips, and key cloud partners"
        },
        "gpu_sales_report": {
            "type": "object",
            "description": "GPU sales performance for the last two years.",
            "properties": {
                "2024": {
                    "type": "object",
                    "description": "GPU sales for 2024",
                    "properties": {
                        "overview": {
                            "type": "string",
                            "description": "An overview of the GPU sales for 2024. 2-3 sentences."
                        },
                        "units_sold": {
                            "type": "integer",
                            "description": "Apprxomiate number of GPUs sold"
                        },
                        "revenue": {
                            "type": "number",
                            "description": "Revenue from GPU sales (USD billions)"
                        }
                    },
                    "required": ["units_sold", "revenue"]
                },
                "2025": {
                    "type": "object",
                    "description": "GPU sales for 2025",
                    "properties": {
                        "overview": {
                            "type": "string",
                            "description": "An overview of the GPU sales for 2025. 2-3 sentences."
                        },
                        "units_sold": {
                            "type": "integer",
                            "description": "Apprxomiate number of GPUs sold"
                        },
                        "revenue": {
                            "type": "number",
                            "description": "Revenue from GPU sales (USD billions)"
                        }
                    },
                    "required": [
                        "revenue",
                        "overview"
                    ]
                },
                "comparative_summary": {
                    "type": "string",
                    "description": "In depth comparison of the GPU salesbetween 2024 and 2025, as well as overall growth and outlook"
                }
            },
            "required": ["2024", "2025", "comparative_summary"]
        }
    },
    "required": [
        "company_name",
        "ceo",
        "cto",
        "gpu_sales_report",
        "compute_and_cloud_offerings"
    ]
}

In [ ]:
headers = {"Authorization": f"Bearer {TAVILY_API_KEY}"}
url = "https://api.tavily.com/research/"

In [ ]:
payload = {
    "input": input,
    "model": model,
    "output_schema": output_schema,
}

result = tavily_client.research(input=input, model=model, output_schema=output_schema)
request_id = result["request_id"]

research_report_structured_output = ""
sources = []

# Poll every 10 seconds
while True:
    response = httpx.get(url + request_id, headers=headers)
    response_json = response.json()

    status = response_json["status"]
    if status == "completed":
        research_report_structured_output = response_json["content"]
        sources = response_json["sources"]
        break

    if status == "failed":
        raise RuntimeError(f"Research failed for {response_json['request_id']}")

    print(f"Status: {status}... polling again in 10 seconds")
    time.sleep(10)

print("Research Complete!")
JSON(research_report_structured_output)

In [ ]:
sources